In [1]:
import pandas as pd

Load dataframes

In [2]:
read = pd.read_feather('Feather/read.feather')

In [3]:
interactions = pd.read_feather('Feather/interactions.feather')

In [4]:
genres = pd.read_feather('Feather/genres.feather')  

In [5]:
books = pd.read_feather('Feather/books.feather')    

In [6]:
reviews = pd.read_feather('Feather/reviews.feather')

In [7]:
user_map = pd.read_feather('Feather/user_map.feather')
book_map = pd.read_feather('Feather/book_map.feather')  
authors = pd.read_feather('Feather/authors.feather')

drop rows in books df where book has no description should i keep this? need consistency so should drop in the other dfs too?

In [8]:
def drop_empty_rows(df, column_name):
   
    df_cleaned = df.dropna(subset=[column_name])
    return df_cleaned

books = drop_empty_rows(books, 'description')

only keep needed rows

In [9]:
interactions = interactions[['user_id', 'book_id', 'review_id', 'is_read', 'rating']]
books = books[['language_code', 'description', 'authors', 'book_id', 'title']]
reviews = reviews[['user_id', 'book_id', 'review_id', 'rating', 'review_text']]

combine list of genre names with the book description

In [10]:
genres = genres.sort_values(by='book_id')
books = books.sort_values(by='book_id')

In [11]:
books = pd.merge(books, genres, on='book_id', how='left')

In [12]:
def combine_description_and_genres(row):
    genres_str = ' '.join(row['genres'])
    return f"{row['description']} {genres_str}"

books['combined_text'] = books.apply(combine_description_and_genres, axis=1)

merge dataframes to get consistent user and book ids

In [13]:
interactions = pd.merge(interactions, user_map, on='user_id', how='left')
interactions = pd.merge(interactions, book_map, on='book_id', how='left')

In [14]:
interactions.drop(columns=['user_id', 'book_id'], inplace=True)
interactions.rename(columns={'user_id_csv': 'user_id', 'book_id_csv': 'book_id'}, inplace=True)

In [15]:
reviews = pd.merge(reviews, user_map, on='user_id', how='left')
reviews = pd.merge(reviews, book_map, on='book_id', how='left')

In [16]:
reviews.drop(columns=['user_id', 'book_id'], inplace=True)
reviews.rename(columns={'user_id_csv': 'user_id', 'book_id_csv': 'book_id'}, inplace=True)

map authors and author ids

In [17]:
books['authors'] = books['authors'].apply(lambda x: [d['author_id'] for d in x])

In [18]:
author_name_mapping = authors.set_index('author_id')['name'].to_dict()

In [19]:
books['authors'] = books['authors'].apply(lambda x: [author_name_mapping[int(author_id)] for author_id in x])

for content i need dataframes: read, books

for collaborative i need interactions and reviews too

save dataframes

In [20]:
read.to_feather('Feather/read.feather')
interactions.to_feather('Feather/interactions.feather')
genres.to_feather('Feather/genres.feather')
books.to_feather('Feather/books.feather')
reviews.to_feather('Feather/reviews.feather')